 # 数据规整化
 数据分析和建模方面大大量编程工作都是用在数据准备上的:加载,清理,转换以及重塑.有时侯,存放在文件或数据库中数据并不能满足你的数据处理应用的要求.pandas和Python标准库提供了一组高级的灵活的,高效的核心函数和算法,它们使你能够轻松地将数据规整化为正确的形式.
 ## 合并数据集
 pandas对象中的数据可以通过一些内置的方式进行合并:
 pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来.SQL或其他关系型数据库的用户对此应该会比较熟悉,因为它实现的就是数据库的连接操作.
 ### 数据库风格的DataFrame合并
 数据集的合并(merge)或连接(join)运算是通过一个或多个键将行链接起来的.这些运算是关系型数据库的核心.pandas的merge函数是对数据应用这些算法的主要切入点.

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series



In [2]:
df1 = DataFrame({'key': list('bbacaab'), 'data1': range(7)})
df2 = DataFrame({'key': list('abd'), 'data2': range(3)})
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2


,key,data2
0,a,0
1,b,1
2,d,2


 这是一种多对一的合并.df1中的数据有多个杯标记为a和b的行,而df2中key列的每个值仅对应一行.对这些对象调用merge即可的到:

In [4]:
pd.merge(df1, df2)


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


 没有指定用那个列进行连接时,merge就会将重叠列的列名作为键.不过最好显式指定一下:

In [5]:
pd.merge(df1, df2, on='key')


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


 如果两个对象的列名不同,也可以分别进行指定:

In [6]:
df3 = DataFrame({'lkey': list('bbacaab'), 'data1': range(7)})
df4 = DataFrame({'rkey': list('abd'), 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


 可能你已经注意到了, 结果里面c和d以及与之相关的数据消失了.默认情况下, merge做的是"inner"连接;结果中的键是交集.其他方式还有"left","right"以及"outer".外连接求取的是键的并集,组合了左连接和右连接的效果:

In [7]:
pd.merge(df1, df2, how='outer')


,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


 多对多的合并操作非常简单,无需额外的工作.

In [8]:
df1 = DataFrame({'key': list('bbacab'), 'data1': range(6)})
df2 = DataFrame({'key': list('ababd'), 'data2': range(5)})
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [9]:
df2


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [10]:
pd.merge(df1, df2, on='key', how='left')


,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


 多对多连接产生的是行的笛卡尔积.由于左边的DataFrame有3个"b"行,右边的有2个,所以最终结果中就有6个"b"行.连接方式只影响出现在结果中的键:

In [11]:
pd.merge(df1, df2, how='inner')


,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


 要根据多个键进行合并,传入一个由列名组成的列表即可:

In [12]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': [
                 'one', 'two', 'one'], 'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': [
                  'one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


 结果中出现哪些键组合取决与所选的合并方式.
 ** 警告 ** : 在进行列 - 列连接时, DataFrame对象中的索引会被丢弃.
 对于合并运算需要考虑的最后一个问题是对重复列名的处理.虽然你可以手工处理列名重叠的问题,但merge有一个更实用的suffixes选项,用于指定附加到左右两个DataFrame对象的重叠列名上的字符串: